### Mapping all Cells with Geopandas and Kepler.GL

In [1]:
import pyodbc
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import asPolygon, box, LineString, Point, Polygon
from geopy.distance import geodesic
from time import time
from keplergl import KeplerGl
import pickle

In [2]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


#### Getting the Data From DBs

In [3]:
null_integer = -9999999999  # with 10 9s
    
cellsConn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                             'Server=10.197.4.184;'
                             'Database=S-iSON;'
                             'UID=mnmureithi;'
                             'PWD=Capri Italy@2019;')

query = (
"Select b1.*, Region,Site_Name,Cell_Name,Latitude,Longitude, Azimuth,BAND from(SELECT a1.starttime,a1.[did],avg([VS.HSDPA.MeanChThroughput]) as 'HSDPA Throughput' ,case when sum([VS.TCP.0]+[VS.TCP.1]+[VS.TCP.2]+[VS.TCP.3]+[VS.TCP.4]+[VS.TCP.5]+[VS.TCP.6]+[VS.TCP.7]+[VS.TCP.8]+[VS.TCP.9]+[VS.TCP.10]+[VS.TCP.11]+[VS.TCP.12]+[VS.TCP.13]+[VS.TCP.14]+[VS.TCP.15]+[VS.TCP.16]+[VS.TCP.17]+[VS.TCP.18]+[VS.TCP.19])=0 then null else sum(((7.5)*[VS.TCP.1])+((12.5)*[VS.TCP.2])+((17.5)*[VS.TCP.3])+((22.5)*[VS.TCP.4])+((27.5)*[VS.TCP.5])+((32.5)*[VS.TCP.6])+((37.5)*[VS.TCP.7])+((42.5)*[VS.TCP.8])+((47.5)*[VS.TCP.9])+((52.5)*[VS.TCP.10])+((57.5)*[VS.TCP.11])+((62.5)*[VS.TCP.12])+((67.5)*[VS.TCP.13])+((72.5)*[VS.TCP.14])+((77.5)*[VS.TCP.15])+((82.5)*[VS.TCP.16])+((87.5)*[VS.TCP.17])+((92.5)*[VS.TCP.18])+((97.5)*[VS.TCP.19]))/sum(([VS.TCP.0]+[VS.TCP.1]+[VS.TCP.2]+[VS.TCP.3]+[VS.TCP.4]+[VS.TCP.5]+[VS.TCP.6]+[VS.TCP.7]+[VS.TCP.8]+[VS.TCP.9]+[VS.TCP.10]+[VS.TCP.11]+[VS.TCP.12]+[VS.TCP.13]+[VS.TCP.14]+[VS.TCP.15]+[VS.TCP.16]+[VS.TCP.17]+[VS.TCP.18]+[VS.TCP.19]))end as 'TCPload' FROM [Z_Hourly_Huawei_Counters].[rnc].[U_HSDPA] as a1 inner join [Z_Hourly_Huawei_Counters].[rnc].[U_TX_RX_Power] as a2 on (a1.did=a2.did and a1.starttime = a2.Starttime) where convert(date,a1.starttime) >= convert(date,getdate()-1) and [VS.HSDPA.MeanChThroughput]<500 group by a1.starttime,a1.[did] ,[VS.HSDPA.MeanChThroughput] union SELECT t1.starttime,t1.[did],case when sum([Active_HS_DSCH_end_user_throughput_DENOM])=0 then Null else avg([Active_HS_DSCH_end_user_throughput_NOM])/avg([Active_HS_DSCH_end_user_throughput_DENOM]) end as 'HSDPA throughput' ,CASE WHEN sum([TX_CR_PWR_CLASS_0_M1000C342] + [TX_CR_PWR_CLASS_1_M1000C343] + [TX_CR_PWR_CLASS_2_M1000C344] + [TX_CR_PWR_CLASS_3_M1000C345] + [TX_CR_PWR_CLASS_4_M1000C346] + [TX_CR_PWR_CLASS_5_M1000C347]+ [TX_CR_PWR_CLASS_6_M1000C348] + [TX_CR_PWR_CLASS_7_M1000C349] + [TX_CR_PWR_CLASS_8_M1000C350] + [TX_CR_PWR_CLASS_9_M1000C351] + [TX_CR_PWR_CLASS_10_M1000C352])= 0 THEN Null ELSE sum(13.58391486*[TX_CR_PWR_CLASS_5_M1000C347]+31.96693704*[TX_CR_PWR_CLASS_6_M1000C348] + 48.63360371*[TX_CR_PWR_CLASS_7_M1000C349] + 61.74514484*[TX_CR_PWR_CLASS_8_M1000C350] + 72.85625595*[TX_CR_PWR_CLASS_9_M1000C351] + 91.36169264*[TX_CR_PWR_CLASS_10_M1000C352])/ sum([TX_CR_PWR_CLASS_0_M1000C342] + [TX_CR_PWR_CLASS_1_M1000C343] + [TX_CR_PWR_CLASS_2_M1000C344] + [TX_CR_PWR_CLASS_3_M1000C345] + [TX_CR_PWR_CLASS_4_M1000C346] + [TX_CR_PWR_CLASS_5_M1000C347]+ [TX_CR_PWR_CLASS_6_M1000C348] + [TX_CR_PWR_CLASS_7_M1000C349] + [TX_CR_PWR_CLASS_8_M1000C350] + [TX_CR_PWR_CLASS_9_M1000C351] + [TX_CR_PWR_CLASS_10_M1000C352]) end as 'TCPload'FROM [KPI_Nokia].[kpi].[UMTS_Main_KPIs] as t1 inner join [Z_Hourly_Nokia_Counters].[nokrww].[Cell_Resource] as t2 on (t1.did=t2.did and t2.starttime = t1.starttime) where convert(date,t1.starttime) >= convert(date,getdate()-1) and case when ([Active_HS_DSCH_end_user_throughput_DENOM])=0 then Null else ([Active_HS_DSCH_end_user_throughput_NOM])/([Active_HS_DSCH_end_user_throughput_DENOM]) end <500 group by t1.[starttime],t1.[did]) as b1 inner join [KPI_Gids].[gid].[KPI_Cell_gids] as b2 on b1.did=b2.did where TCPload<70"
)

In [4]:
def setting_cells_df_datatypes(cells_df):
    cells_df.starttime = pd.to_datetime(cells_df.starttime)

    cells_df.Azimuth = cells_df.Azimuth.astype(float)
    
    # Removing the special characters from the name Columns
    # Repeating it might be faster than using a loop TODO: Confirm if this is true
    cells_df['Cell_Name'] = cells_df['Cell_Name'].str.translate({ord(c): " " for c in "!@#$%^&*()[]{};:,./<>?\|`~=+"})
    cells_df['Site_Name'] = cells_df['Site_Name'].str.translate({ord(c): " " for c in "!@#$%^&*()[]{};:,./<>?\|`~=+"})
    

    return cells_df

In [5]:
cells_df = setting_cells_df_datatypes(pd.read_sql(query, cellsConn))

In [6]:
cells_df['starttime']=cells_df['starttime'].astype(str)

In [7]:
cells_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129123 entries, 0 to 129122
Data columns (total 11 columns):
starttime           129123 non-null object
did                 129123 non-null int64
HSDPA Throughput    129123 non-null float64
TCPload             129123 non-null float64
Region              129122 non-null object
Site_Name           129123 non-null object
Cell_Name           129123 non-null object
Latitude            125348 non-null float64
Longitude           125348 non-null float64
Azimuth             125348 non-null float64
BAND                126721 non-null object
dtypes: float64(5), int64(1), object(5)
memory usage: 10.8+ MB


In [8]:
cells_df.shape

(129123, 11)

In [9]:
#cells_df.Main_Frequency = cells_df.Main_Frequency.fillna(null_integer)#.astype(int)

In [10]:
#cells_df.Main_Frequency=cells_df.Main_Frequency.astype(float)

#### Creating Shapely Polygons

In [11]:
technology_bands = ['LTE800','GSM900','U900-C1', 'U900-C2','GSM1800','LTE1800', 'U2100-C1', 'U2100-C2']  # sorted(list(self.cells.BAND.unique()))  # several bands (8)
technology_list = ['GSM', 'LTE', 'UMTS']  # sorted(list(self.cells.TECHNOLOGY.unique()))  # 3 items
radius_list = [200, 190, 180, 170, 110, 100, 80, 70]


def get_radius_tech_color(band):
    '''
        Takes the band of a site or cell and returns the color style, the radius of the cone and the folder
    :param band:
    :return:
    '''

    try:
        tech_index = technology_bands.index(band)
        radius = radius_list[tech_index]
    except:
        tech_index = 2  #TODO Refine this: Tech is most probably UMTS
        radius = 0  # We Ignore Radius


    folder_index = 0
    if tech_index==1 or tech_index==4:  # Getting the folder index for the kml file being generated
        folder_index = 0
    elif tech_index==0 or tech_index==5 :
        folder_index = 1
    else:
        folder_index = 2

    return radius, folder_index

In [12]:
cells_df = cells_df.loc[~pd.isnull(cells_df.Latitude)]
cells_df = cells_df.loc[~pd.isnull(cells_df.Longitude)]
cells_df['SITE_LONGITUDE_LATITUDE'] = list(zip(cells_df.Longitude, cells_df.Latitude))
cells_df['Radius_Tech'] = [get_radius_tech_color(band) for band in cells_df['BAND']]#List comprehension
cells_df['Cell_degrees']=np.degrees(np.radians(90)-np.radians(cells_df['Azimuth']))

# Shapely Point for the Site Location
# point_polygons = [Point(site_lat_long) for site_lat_long in cells_df['SITE_LATITUDE_LONGITUDE']]

# Shapely Pie for Individual Cells
start = time()
# Removing the reversal of the coords coz I'm not using SimpleKml [::-1]
pie_polygons = [Polygon([longitude_latitude] + 
                        [tuple(
                            geodesic(meters=polygon_tech[0]).destination(point=longitude_latitude, bearing=bearing))[:-1] #geodesic function returns(lon,lat,alt)
                         for bearing in
     np.arange(Cell_degrees-30, Cell_degrees+30, 6)] 
                        + [longitude_latitude]) 
                for (longitude_latitude, polygon_tech, Cell_degrees) in cells_df[['SITE_LONGITUDE_LATITUDE', 'Radius_Tech', 'Cell_degrees']].values]

print(time() - start)
# Polygon()

273.91165947914124


In [13]:
cells_df.head()

,starttime,did,HSDPA Throughput,TCPload,Region,Site_Name,Cell_Name,Latitude,Longitude,Azimuth,BAND,SITE_LONGITUDE_LATITUDE,Radius_Tech,Cell_degrees
0,2020-02-12 01:00:00,34690,254.108002,21.150811,NAIROBI WEST,14713_NW_NI2425-Rose_Avenue_OUTH_MRT,14713_NW_NI4425-Rose_Avenue_OUTH_U900-4,-1.29389,36.795700,140.0,U900-C2,"(36.79570007324219, -1.2938899993896484)","(170, 2)",-50.0
1,2020-02-12 11:00:00,34690,367.656982,50.836575,NAIROBI WEST,14713_NW_NI2425-Rose_Avenue_OUTH_MRT,14713_NW_NI4425-Rose_Avenue_OUTH_U900-4,-1.29389,36.795700,140.0,U900-C2,"(36.79570007324219, -1.2938899993896484)","(170, 2)",-50.0
2,2020-02-11 03:00:00,68914,349.995483,17.817500,NAIROBI EAST,14774_NE_NI4532-Nyayo_Stadium_OUTE_MGF,14774_NE_NI4532-Nyayo_Stadium_OUTH_U900-0,-1.30341,36.825401,10.0,U900-C1,"(36.825401306152344, -1.3034100532531738)","(180, 2)",80.0
3,2020-02-11 00:00:00,68914,363.704498,19.321565,NAIROBI EAST,14774_NE_NI4532-Nyayo_Stadium_OUTE_MGF,14774_NE_NI4532-Nyayo_Stadium_OUTH_U900-0,-1.30341,36.825401,10.0,U900-C1,"(36.825401306152344, -1.3034100532531738)","(180, 2)",80.0
4,2020-02-11 01:00:00,68914,199.235992,18.317177,NAIROBI EAST,14774_NE_NI4532-Nyayo_Stadium_OUTE_MGF,14774_NE_NI4532-Nyayo_Stadium_OUTH_U900-0,-1.30341,36.825401,10.0,U900-C1,"(36.825401306152344, -1.3034100532531738)","(180, 2)",80.0


#### Creating the GeoPandas Dataframe

In [14]:
gpd_cells = gpd.GeoDataFrame(cells_df, geometry=pie_polygons)
gpd_cells.crs = {'init':'epsg:4326'}

In [15]:
gpd_cells['HSDPA Throughput'].fillna(0, inplace=True)
gpd_cells['TCPload'].fillna(0, inplace=True)

In [16]:
gpd_cells.head()

,starttime,did,HSDPA Throughput,TCPload,Region,Site_Name,Cell_Name,Latitude,Longitude,Azimuth,BAND,SITE_LONGITUDE_LATITUDE,Radius_Tech,Cell_degrees,geometry
0,2020-02-12 01:00:00,34690,254.108002,21.150811,NAIROBI WEST,14713_NW_NI2425-Rose_Avenue_OUTH_MRT,14713_NW_NI4425-Rose_Avenue_OUTH_U900-4,-1.29389,36.795700,140.0,U900-C2,"(36.79570007324219, -1.2938899993896484)","(170, 2)",-50.0,"POLYGON ((36.79570 -1.29389, 36.79597 -1.29577..."
1,2020-02-12 11:00:00,34690,367.656982,50.836575,NAIROBI WEST,14713_NW_NI2425-Rose_Avenue_OUTH_MRT,14713_NW_NI4425-Rose_Avenue_OUTH_U900-4,-1.29389,36.795700,140.0,U900-C2,"(36.79570007324219, -1.2938899993896484)","(170, 2)",-50.0,"POLYGON ((36.79570 -1.29389, 36.79597 -1.29577..."
2,2020-02-11 03:00:00,68914,349.995483,17.817500,NAIROBI EAST,14774_NE_NI4532-Nyayo_Stadium_OUTE_MGF,14774_NE_NI4532-Nyayo_Stadium_OUTH_U900-0,-1.30341,36.825401,10.0,U900-C1,"(36.825401306152344, -1.3034100532531738)","(180, 2)",80.0,"POLYGON ((36.82540 -1.30341, 36.82644 -1.30186..."
3,2020-02-11 00:00:00,68914,363.704498,19.321565,NAIROBI EAST,14774_NE_NI4532-Nyayo_Stadium_OUTE_MGF,14774_NE_NI4532-Nyayo_Stadium_OUTH_U900-0,-1.30341,36.825401,10.0,U900-C1,"(36.825401306152344, -1.3034100532531738)","(180, 2)",80.0,"POLYGON ((36.82540 -1.30341, 36.82644 -1.30186..."
4,2020-02-11 01:00:00,68914,199.235992,18.317177,NAIROBI EAST,14774_NE_NI4532-Nyayo_Stadium_OUTE_MGF,14774_NE_NI4532-Nyayo_Stadium_OUTH_U900-0,-1.30341,36.825401,10.0,U900-C1,"(36.825401306152344, -1.3034100532531738)","(180, 2)",80.0,"POLYGON ((36.82540 -1.30341, 36.82644 -1.30186..."


In [17]:
# Pickling the pickle file
with open("D:\\Python scripts\\pickled\\filtered_UMTS_data.pkl", "wb") as fp:   #Pickling
    pickle.dump(gpd_cells, fp)

In [18]:
#Unpickling the pickle file
with open("D:\\Python scripts\\pickled\\filtered_UMTS_data.pkl", "rb") as fp:   # Unpickling
      gpd_cells = pickle.load(fp)

In [19]:
gpd_cells.head()

,starttime,did,HSDPA Throughput,TCPload,Region,Site_Name,Cell_Name,Latitude,Longitude,Azimuth,BAND,SITE_LONGITUDE_LATITUDE,Radius_Tech,Cell_degrees,geometry
0,2020-02-12 01:00:00,34690,254.108002,21.150811,NAIROBI WEST,14713_NW_NI2425-Rose_Avenue_OUTH_MRT,14713_NW_NI4425-Rose_Avenue_OUTH_U900-4,-1.29389,36.795700,140.0,U900-C2,"(36.79570007324219, -1.2938899993896484)","(170, 2)",-50.0,"POLYGON ((36.79570 -1.29389, 36.79597 -1.29577..."
1,2020-02-12 11:00:00,34690,367.656982,50.836575,NAIROBI WEST,14713_NW_NI2425-Rose_Avenue_OUTH_MRT,14713_NW_NI4425-Rose_Avenue_OUTH_U900-4,-1.29389,36.795700,140.0,U900-C2,"(36.79570007324219, -1.2938899993896484)","(170, 2)",-50.0,"POLYGON ((36.79570 -1.29389, 36.79597 -1.29577..."
2,2020-02-11 03:00:00,68914,349.995483,17.817500,NAIROBI EAST,14774_NE_NI4532-Nyayo_Stadium_OUTE_MGF,14774_NE_NI4532-Nyayo_Stadium_OUTH_U900-0,-1.30341,36.825401,10.0,U900-C1,"(36.825401306152344, -1.3034100532531738)","(180, 2)",80.0,"POLYGON ((36.82540 -1.30341, 36.82644 -1.30186..."
3,2020-02-11 00:00:00,68914,363.704498,19.321565,NAIROBI EAST,14774_NE_NI4532-Nyayo_Stadium_OUTE_MGF,14774_NE_NI4532-Nyayo_Stadium_OUTH_U900-0,-1.30341,36.825401,10.0,U900-C1,"(36.825401306152344, -1.3034100532531738)","(180, 2)",80.0,"POLYGON ((36.82540 -1.30341, 36.82644 -1.30186..."
4,2020-02-11 01:00:00,68914,199.235992,18.317177,NAIROBI EAST,14774_NE_NI4532-Nyayo_Stadium_OUTE_MGF,14774_NE_NI4532-Nyayo_Stadium_OUTH_U900-0,-1.30341,36.825401,10.0,U900-C1,"(36.825401306152344, -1.3034100532531738)","(180, 2)",80.0,"POLYGON ((36.82540 -1.30341, 36.82644 -1.30186..."


In [20]:
gpd_cells=gpd_cells.head(50000)

####  Mapping Cells with Kepler.gl

In [21]:
cell_map = KeplerGl(height=800)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


In [22]:
cell_map.add_data(data=gpd_cells, name="Saf_Cells")

In [23]:
cell_map

KeplerGl(data={'Saf_Cells': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2…

In [24]:
config=cell_map.config

In [25]:
# Pickling the pickle file
with open("D:\\Python scripts\\pickled\\filtered_UMTS_map.pkl", "wb") as fp:   #Pickling
    pickle.dump(config, fp)

In [26]:
#Unpickling the pickle file
with open("D:\\Python scripts\\pickled\\filtered_UMTS_map.pkl", "rb") as fp:   # Unpickling
      config = pickle.load(fp)
#fp.close()

In [27]:
#cell_map.save_to_html(file_name="cell_map3G.html")
# this will save map with provided data and config
cell_map.save_to_html(config=config, file_name='3G_map.html')

Map saved to 3G_map.html!


In [28]:
cell_map = KeplerGl(height=800,config=config)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


In [29]:
cell_map

KeplerGl(height=800)

In [30]:
cell_map.add_data(data=gpd_cells, name="Saf_Cells")